## Projeto 3 - Machine Learning

Caio Emanuel Rhoden - RA 214129

In [1]:
from pacman import runGames,readCommand
from game import Agent, Directions
from util import manhattanDistance, PriorityQueue
import random
import statistics

### Sumário

### Cromossomo

In [2]:
def create_random_chromossome():

    chromossome = {
        #no near ghost
        'explorer': PriorityQueue(),

        #enemy coming from south - ghost in same x and higher y
        'danger_1': PriorityQueue(),

        #enemy coming from south and east - ghost in higher x and higher y
        'danger_2': PriorityQueue(),

        #enemy coming from east, - ghost higher x and same y
        'danger_3': PriorityQueue(),

        #enemy coming from north east - ghost higher x and lower y
        'danger_4':PriorityQueue(),

        #enemy coming from north - ghost at same x and lower y
        'danger_5':PriorityQueue(),

        #enemy coming from north and west - ghost lower x and lower y
        'danger_6': PriorityQueue(),

        #enemy coming from west - ghost lower x and same y
        'danger_7':PriorityQueue(),

        #enemy coming from south and west - ghost lower x and higher y
        'danger_8': PriorityQueue()

    }
    
    for i in range(8):
        values = [0, 1, 2, 3]
        random.shuffle(values)
        priorities = [0, 1, 2, 3]
        random.shuffle(priorities)

        chromossome[list(chromossome.keys())[i]].push(values[0], priorities[0])
        chromossome[list(chromossome.keys())[i]].push(values[1], priorities[1])
        chromossome[list(chromossome.keys())[i]].push(values[2], priorities[2])
        chromossome[list(chromossome.keys())[i]].push(values[0], priorities[3])
        
    
    return chromossome

In [3]:
a = create_random_chromossome()['explorer']
print(a.heap)
print(a.pop())
print(a.heap)
print(a.pop())
print(a.heap)
print(a.pop())
print(a.heap)

[(0, 1, 2), (1, 3, 3), (2, 2, 0), (3, 0, 3)]
2
[(1, 3, 3), (3, 0, 3), (2, 2, 0)]
3
[(2, 2, 0), (3, 0, 3)]
0
[(3, 0, 3)]


### Agente

In [4]:
class SingleLookAgent(Agent):
    "Define some rules to the agent behaviour"
    def __init__(self, dist, chromo, index=0):
        super().__init__(index)
        self.chromossome = chromo
        self.dist = dist
    
    def goLeft(self, current):
        return Directions.LEFT[current]
    def goRight(self, current):
        return Directions.RIGHT[current]
    def goReverse(self, current):
        return Directions.REVERSE[current]
    def goForward(self, current):
        return current
    
    def getActionfromQueue(self, state, current, mode):
        actionsPushed = []
        found = False
        
        actionsList = ['LEFT', 'RIGHT', 'REVERSE', 'FORWARD']
        
        actionsDict = {
            'LEFT': self.goLeft(current),
            'RIGHT': self.goRight(current),
            'REVERSE': self.goReverse(current),
            'FORWARD': self.goForward(current)
        }
        while not found and len((self.chromossome[mode].heap)) > 0:
            actionsPushed.append(self.chromossome[mode].pop())
            action = actionsDict[actionsList[actionsPushed[-1]]]
            if action in  state.getLegalPacmanActions() and action != 'Stop':
                found = True
        for i in range(len(actionsPushed)):
            self.chromossome[mode].push(i, actionsPushed[i])
        if found:
            return action
        else:
            return 'Stop'

    def getAction(self, state):
        
        pacman_pos = state.getPacmanPosition()
        dist_ghost_1 = manhattanDistance(pacman_pos , state.getGhostPosition(1))
        dist_ghost_2 = manhattanDistance(pacman_pos , state.getGhostPosition(2))
        
        if dist_ghost_1 <= dist_ghost_2:
            near_dist = dist_ghost_1
            near_ghost = state.getGhostPosition(1)
        else:
            near_ghost = state.getGhostPosition(2)
            near_dist = dist_ghost_2
        near_dist = 100
        current = state.getPacmanState().configuration.direction
        
        #defines random init between legal actions
        if current == 'Stop':
            return state.getLegalPacmanActions()[random.randrange(len(state.getLegalPacmanActions()))]
            
        elif near_dist > self.dist:
            return self.getActionfromQueue(state, current, 'explorer')
            
        else:
            if pacman_pos[0] == near_ghost[0] and pacman_pos[1] < near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_1')
            
            elif pacman_pos[0] < near_ghost[0] and pacman_pos[1] < near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_2')
            
            elif pacman_pos[0] < near_ghost[0] and pacman_pos[1] == near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_3')
            
            elif pacman_pos[0] < near_ghost[0] and pacman_pos[1] > near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_4')
            
            elif pacman_pos[0] == near_ghost[0] and pacman_pos[1] > near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_5')
            
            elif pacman_pos[0] > near_ghost[0] and pacman_pos[1] > near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_6')
            
            elif pacman_pos[0] > near_ghost[0] and pacman_pos[1] == near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_7')
            
            elif pacman_pos[0] > near_ghost[0] and pacman_pos[1] < near_ghost[1]:
                return self.getActionfromQueue(state, current, 'danger_8')
            else:
                print("NO CONDITION - CURRENT")
                return current
    

In [5]:

agente = SingleLookAgent(5, create_random_chromossome())
args = readCommand(['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic','-n', '3'])
args['pacman'] = agente
a = runGames(**args)
print(a)


['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '3']
[<game.Game object at 0x000001BEBD4D49A0>, <game.Game object at 0x000001BEBD4D4FD0>, <game.Game object at 0x000001BEBD4D4D60>]


In [6]:
print(a[0].state.isWin())

False


### Initiate Random Population

In [7]:
def randomPopulation(size):
    population = []
    for i in range(size):
        population.append(create_random_chromossome())
        
    return population

In [8]:
randomPopulation(2)

[{'explorer': <util.PriorityQueue at 0x1bebd40a520>,
  'danger_1': <util.PriorityQueue at 0x1bebd40a4c0>,
  'danger_2': <util.PriorityQueue at 0x1bebd3fa940>,
  'danger_3': <util.PriorityQueue at 0x1bebd3fa790>,
  'danger_4': <util.PriorityQueue at 0x1bebd3fa910>,
  'danger_5': <util.PriorityQueue at 0x1bebd3fa5e0>,
  'danger_6': <util.PriorityQueue at 0x1bebd3fa0a0>,
  'danger_7': <util.PriorityQueue at 0x1bebd4dd4f0>,
  'danger_8': <util.PriorityQueue at 0x1bebd4ddd00>},
 {'explorer': <util.PriorityQueue at 0x1bebd4ddbe0>,
  'danger_1': <util.PriorityQueue at 0x1bebd4ddd90>,
  'danger_2': <util.PriorityQueue at 0x1bebd4dd4c0>,
  'danger_3': <util.PriorityQueue at 0x1bebd4ddd30>,
  'danger_4': <util.PriorityQueue at 0x1bebd4dddc0>,
  'danger_5': <util.PriorityQueue at 0x1bebd4dd400>,
  'danger_6': <util.PriorityQueue at 0x1bebd4dd910>,
  'danger_7': <util.PriorityQueue at 0x1bebd4ddb50>,
  'danger_8': <util.PriorityQueue at 0x1bebd4ddd60>}]

### Fitness Function

Aqui vamos estabelecer como condição de sucesso para os individuos, inicialmente podemos estabelecer como critério de sucesso a vitória para os tabuleiros, e para avaliar a aptidão dos individuos até lá, a pontuação total obtida. A ideia inicial era fazer  a aptidão a partir do número de comidas que o nosso agente consumiu, no entando por não conseguir extrair essa informação da engine foi mantido o score médio dos jogos

In [9]:
def fitnessFunction(gamesResult):
    gamesScores = [i.state.getScore() for i in gamesResult]
    avgScore = statistics.mean(gamesScores)            
    return(avgScore)

### Seleção

Para seleção aplicaremos o método de campeonado em que faremos n campeonatos, dependendo do número de individuos que queremos selecionar, e para cada campeonato pegaremos um número aleatorio de individuos, e dentre esses selecionaremos o com o maior fitness score para ser um dos sortudos que vão passar para a próxima geração

In [47]:
def selection(population, n):
    selected = []
    
    popScores = []
    for i in range(len(population)):
        agente =  SingleLookAgent(5, population[i])
        args = readCommand(['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic','-n', '1'])
        args['pacman'] = agente
        popScores.append(runGames(**args))
    
    fitnessScores = [fitnessFunction(i) for i in popScores]
    
    totalScore = sum(fitnessScores)
    for i in range(n):
        max_value = max(random.sample(fitnessScores, random.randrange(1,int(max(len(population)//4,2)))))
        selected.append(population[fitnessScores.index(max_value)])
        
    return selected, popScores, fitnessScores

In [48]:
pop = randomPopulation(5)
args = readCommand(['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic','-n', '3'])
selection(pop,1)

['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '3']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']


([{'explorer': <util.PriorityQueue at 0x1bebd82cc40>,
   'danger_1': <util.PriorityQueue at 0x1bebd82cf70>,
   'danger_2': <util.PriorityQueue at 0x1bebd82caf0>,
   'danger_3': <util.PriorityQueue at 0x1bebd82c520>,
   'danger_4': <util.PriorityQueue at 0x1bebd82cac0>,
   'danger_5': <util.PriorityQueue at 0x1bebd82cbe0>,
   'danger_6': <util.PriorityQueue at 0x1bebd82c130>,
   'danger_7': <util.PriorityQueue at 0x1bebd82ce80>,
   'danger_8': <util.PriorityQueue at 0x1bebd82c9a0>}],
 [[<game.Game at 0x1bebd64c070>],
 [-494.0, -479.0, -544.0, -495.0, -535.0])

### Crossover

In [49]:
def reproduction(parents, rate, newGenSize):
    
    modes = ['explorer', 'danger_1', 'danger_2', 'danger_3', 'danger_4', 'danger_5', 'danger_6', 'danger_7', 'danger_8']
    newGen = [] 
    
    for j in range(int(rate*newGenSize)//1):
        
        
        p1 = random.randrange(len(parents))
        p2 = p1
        while p2 == p1:
            p2 = random.randrange(len(parents))
        
        parent1 = parents[p1]
        parent2 = parents[p2]
        
        sep1 = random.randrange(7)
        sep2 = random.randrange(sep1+1, 8)
        new_gene = create_random_chromossome()
        for i in range(9):
            if i <= sep1:
                new_gene[modes[i]] = parent1[modes[i]]
            elif i > sep1 and i <= sep2:
                new_gene[modes[i]] = parent2[modes[i]]
            else:
                new_gene[modes[i]] = parent1[modes[i]]
        
        newGen.append(new_gene)
    
    
    for i in range(newGenSize - int((rate*newGenSize)//1)):
        newGen.append(parents[random.randrange(len(parents))])
    
    return newGen
    

In [50]:
example = randomPopulation(2)
print(example)
print()
print()
print(reproduction(example, 1, 2))

[{'explorer': <util.PriorityQueue object at 0x000001BEBD64C6A0>, 'danger_1': <util.PriorityQueue object at 0x000001BEBD64C520>, 'danger_2': <util.PriorityQueue object at 0x000001BEBD64CAC0>, 'danger_3': <util.PriorityQueue object at 0x000001BEBD64CCA0>, 'danger_4': <util.PriorityQueue object at 0x000001BEBD64CA60>, 'danger_5': <util.PriorityQueue object at 0x000001BEBD64C700>, 'danger_6': <util.PriorityQueue object at 0x000001BEBD6CC8E0>, 'danger_7': <util.PriorityQueue object at 0x000001BEBD6CC490>, 'danger_8': <util.PriorityQueue object at 0x000001BEBD6CC670>}, {'explorer': <util.PriorityQueue object at 0x000001BEBD6CC5B0>, 'danger_1': <util.PriorityQueue object at 0x000001BEBD6CCBB0>, 'danger_2': <util.PriorityQueue object at 0x000001BEBD6CC550>, 'danger_3': <util.PriorityQueue object at 0x000001BEBD6CC7C0>, 'danger_4': <util.PriorityQueue object at 0x000001BEBD970310>, 'danger_5': <util.PriorityQueue object at 0x000001BEBD970B80>, 'danger_6': <util.PriorityQueue object at 0x000001B

### Mutação

Para mutação foi optado pelo swap entre as PriorityQueues de uma quantidade da população definida pelo argumento rate

In [51]:
def mutation(population, rate):
    population_len = [i for i in range(len(population))]
    mutated = random.sample(population_len, int((rate * len(population))// 1))
    modes = ['explorer', 'danger_1', 'danger_2', 'danger_3', 'danger_4', 'danger_5', 'danger_6', 'danger_7', 'danger_8']
    
    
    for i in mutated:
        mut_1 = random.randrange(9)
        mut_2 = mut_1
        while mut_1 == mut_2:
            mut_2 = random.randrange(9)
        saved = population[i][modes[mut_1]]
        population[i][modes[mut_1]] = population[i][modes[mut_2]]
        population[i][modes[mut_2]] = saved
    
    return population 
        
        

In [52]:
example = randomPopulation(3)
print(example[0])
print(mutation(example,0.5)[0])

{'explorer': <util.PriorityQueue object at 0x000001BEBDA56700>, 'danger_1': <util.PriorityQueue object at 0x000001BEBDA56F40>, 'danger_2': <util.PriorityQueue object at 0x000001BEBDA56820>, 'danger_3': <util.PriorityQueue object at 0x000001BEBDA56430>, 'danger_4': <util.PriorityQueue object at 0x000001BEBDA56580>, 'danger_5': <util.PriorityQueue object at 0x000001BEBDA565B0>, 'danger_6': <util.PriorityQueue object at 0x000001BEBDA56910>, 'danger_7': <util.PriorityQueue object at 0x000001BEBDA56490>, 'danger_8': <util.PriorityQueue object at 0x000001BEBD3FA4C0>}
{'explorer': <util.PriorityQueue object at 0x000001BEBDA56700>, 'danger_1': <util.PriorityQueue object at 0x000001BEBDA56F40>, 'danger_2': <util.PriorityQueue object at 0x000001BEBDA56820>, 'danger_3': <util.PriorityQueue object at 0x000001BEBDA56430>, 'danger_4': <util.PriorityQueue object at 0x000001BEBDA56580>, 'danger_5': <util.PriorityQueue object at 0x000001BEBDA565B0>, 'danger_6': <util.PriorityQueue object at 0x000001BEB

### Treinamento

In [53]:
def train(maxGen,populationSize, mutationRate, crossoverRate, selectionSize, newGenSize):
    
    population = randomPopulation(populationSize)
    for i in range(maxGen):
        newPopulation = []
        parents, gamesResult, scores = selection(population, selectionSize)
        print(f"Best Score Gen {i}: {max(scores)}")
        newGen = reproduction(parents, crossoverRate, newGenSize)
        cutOffScore = sorted(scores)[populationSize-newGenSize]
        for i in range(populationSize):
            if scores[i] < cutOffScore:
                newGene = newGen.pop()
                newPopulation.append(newGene)
            else:
                newPopulation.append(population[i])
        population = mutation(newPopulation, mutationRate)
    
    return population
                

In [56]:
train(100, 20, 0.05, 0.8, 10, 15)

['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent', '-l', 'smallClassic', '-n', '1']
['-q', '--pacman', 'LeftTurnAgent'

[{'explorer': <util.PriorityQueue at 0x1bebd673ca0>,
  'danger_1': <util.PriorityQueue at 0x1bebd645850>,
  'danger_2': <util.PriorityQueue at 0x1bebd57d7f0>,
  'danger_3': <util.PriorityQueue at 0x1bebd673b50>,
  'danger_4': <util.PriorityQueue at 0x1bebd673d00>,
  'danger_5': <util.PriorityQueue at 0x1bebd645430>,
  'danger_6': <util.PriorityQueue at 0x1bebd6bd520>,
  'danger_7': <util.PriorityQueue at 0x1bebd673d00>,
  'danger_8': <util.PriorityQueue at 0x1bebd645250>},
 {'explorer': <util.PriorityQueue at 0x1bebd673ca0>,
  'danger_1': <util.PriorityQueue at 0x1bebd645850>,
  'danger_2': <util.PriorityQueue at 0x1bebd57d7f0>,
  'danger_3': <util.PriorityQueue at 0x1bebd673b50>,
  'danger_4': <util.PriorityQueue at 0x1bebd673d00>,
  'danger_5': <util.PriorityQueue at 0x1bebd645430>,
  'danger_6': <util.PriorityQueue at 0x1bebd6bd520>,
  'danger_7': <util.PriorityQueue at 0x1bebd673d00>,
  'danger_8': <util.PriorityQueue at 0x1bebd645250>},
 {'explorer': <util.PriorityQueue at 0x1bebd

### Treinamento -  Small Classic

In [18]:
initial_population = randomPopulation(20)


### Treinamento -  Medium Classic

### Treinamento -  Original Classic

### Avaliação Genetic Programming